In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K

In [ ]:
# Import required libraries

import cv2                 # working with, mainly resizing, images
import numpy as np
import os
from random import shuffle # randomizing ordered data
from tqdm import tqdm      # a nice pretty percentage bar for tasks.


In [ ]:
# define variables 

TRAIN_DIR = 'data/train'
TEST_DIR = 'data/test'
IMG_WIDTH, IMG_HEIGHT = 100, 100 # dimensions of our images.
LR = 1e-3

MODEL_NAME = 'pizzas-{}-{}.model'.format(LR, '2conv-basic') # just so we remember which saved model is which, sizes must match

In [ ]:
# One hot encoding of the target colum
def label_img(img, word_label):
    if word_label == 'Pepperoni': return [1,0]
    elif word_label == 'Sausage': return [0,1]

In [ ]:
# Create training data
def create_train_data(directory='', label='', num_of_images=1000):
    training_data = []
    if directory=='':
        directory = TRAIN_DIR
    label_dir = directory+'/'+label
    i=0
    for img in tqdm(os.listdir(label_dir)):
        img_label = label_img(img, label)
        path = os.path.join(label_dir,img)
        img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (IMG_WIDTH, IMG_HEIGHT))
        training_data.append([np.array(img),np.array(img_label)])
        i=i+1
        if i>num_of_images:
            break
    return training_data

In [ ]:
train_data_Pepperoni = create_train_data('Pepperoni')
train_data_Sausage = create_train_data('Sausage')
train_data = train_data_Pepperoni.append(train_data_Sausage)

In [ ]:
train_data = train_data_Pepperoni

In [ ]:
shuffle(train_data)

In [ ]:
np.save('data/train_data.npy', train_data)

In [ ]:
train_data

In [ ]:
shuffle(training_data)
np.save('data/train_data.npy', training_data)

# Data Augumentation


In [ ]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img


In [ ]:
# Define the data generation object
datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.4,
        height_shift_range=0.4,
        shear_range=0.25,
        zoom_range=0.25,
        rescale=1.01,
        zca_epsilon=1e-6,
        horizontal_flip=True,
        fill_mode='nearest')


In [ ]:
os.getcwd()

In [ ]:
string="data/new_train/Pepperoni/._0_9746.jpeg"
print (string[0:-1])

In [ ]:
# Function to generate images
def augument_images(source_dir_name, dest_dir_name, num_images):
    for image in tqdm(os.listdir(source_dir_name)):
        img = load_img(source_dir_name+'/'+image)
        x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
        x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)
        # the .flow() command below generates batches of randomly transformed images
        # and saves the results to the `preview/` directory
        i = 0
        image_name = image[0:-4]
        for batch in datagen.flow(x, batch_size=1,
                          save_to_dir=dest_dir_name, save_prefix=image_name, save_format='jpeg'):
            i += 1
            if i > num_images:
                break  # otherwise the generator would loop indefinitely


In [ ]:
#source_dir_name = '/home/datascience/projects/vsoft/data/test'
pepperoni_source_dir_name = '/home/datascience/projects/vsoft/data/train/Pepperoni'
pepperoni_dest_dir_name = 'data/new_train/Pepperoni/'

In [ ]:
augument_images(pepperoni_source_dir_name, pepperoni_dest_dir_name, 4)

In [ ]:
#source_dir_name = '/home/datascience/projects/vsoft/data/test'
sausage_source_dir_name = '/home/datascience/projects/vsoft/data/train/Sausage'
sausage_dest_dir_name = '/home/datascience/projects/vsoft/data/new_train/Sausage'

In [ ]:
augument_images(sausage_source_dir_name,sausage_dest_dir_name,10)

# Data Preperation

In [ ]:
source_dir_name = '/home/datascience/projects/vsoft/data/new_train'

In [ ]:
pepperoni_data = create_train_data(source_dir_name,'Pepperoni',1000)

In [ ]:
sausage_data = create_train_data(source_dir_name,'Sausage',1000)

## Save data for later use

In [ ]:
np.save('data/pepperoni_data.npy', pepperoni_data)

In [ ]:
np.save('data/sausage_data.npy', sausage_data)

## Segregate the data into train and test

In [ ]:
pepperoni_data = np.load('data/sausage_data.npy')
sausage_data = np.load('data/sausage_data.npy')

In [ ]:
full_data.shape

In [ ]:
full_data = np.concatenate((pepperoni_data, sausage_data), axis=0)

In [ ]:
shuffle(full_data)

In [ ]:
train = full_data[:-1600]
test = full_data[-400:]

In [ ]:
np.array(train).shape

In [ ]:
X=[]
Y=[]

In [ ]:
for i in train:
    #print(i[0])
    #print(len(i))
    X.append(i[0].reshape(IMG_WIDTH, IMG_HEIGHT,1))
    Y.append(i[1])
    #print(i[0].reshape(IMG_WIDTH, IMG_HEIGHT,1).shape)

In [ ]:
np.array(X).shape
np.array(Y).shape

In [ ]:
X= np.array(X)
Y= np.array(Y)

In [ ]:
X_test=[]
Y_test=[]

In [ ]:
len(train)

In [ ]:
for i in test:
    #print(i[0])
    #print(len(i))
    X_test.append(i[0].reshape(IMG_WIDTH, IMG_HEIGHT,1))
    #print(i[0].reshape(IMG_WIDTH, IMG_HEIGHT,1).shape)

In [ ]:
np.array(X_test).shape

In [ ]:
X_test = np.array(X_test)

In [ ]:
X = np.array([i[0] for i in train[0:600]]).reshape(-1,IMG_WIDTH,IMG_HEIGHT,1)
Y = np.array([i[1] for i in train[0:600]])

test_x = np.array([i[0] for i in test[0:90]]).reshape(-1,IMG_WIDTH,IMG_HEIGHT,1)
test_y = [i[1] for i in test[0:90]]

# Convolution Model

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
import tensorflow as tf

In [ ]:
K.clear_session()
sess = tf.Session()
K.set_session(sess)

In [ ]:
# Define variables
nb_train_samples = 2000
nb_validation_samples = 800
epochs = 50
batch_size = 16
num_classes = 2

if K.image_data_format() == 'channels_first':
    input_shape = (1, IMG_WIDTH, IMG_HEIGHT) # For RGB, (3, IMG_WIDTH, IMG_HEIGHT) 
else:
    input_shape = (IMG_WIDTH, IMG_HEIGHT, 1) # For RGB, (IMG_WIDTH, IMG_HEIGHT, 3) 



In [ ]:
# GPU DEFAULT

model = Sequential()
model.add(Conv2D(32, kernel_size=(5, 5), strides=(1, 1),
                 activation='relu',
                 input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Conv2D(64, (5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

In [ ]:
# CPU Code
with tf.device('/cpu:0'):
    #x = tf.placeholder(tf.float32, shape=(None, 784))
    # shared model living on CPU:0
    # it won't actually be run during training; it acts as an op template
    # and as a repository for shared variables
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(5, 5), strides=(1, 1),
                 activation='relu',
                 input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Conv2D(64, (5, 5), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(100, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))

In [ ]:
# Compile the model
model.compile('adam', 'categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Fit the model
history = model.fit(X, Y, verbose=0, epochs=1)